# Demo RAG Pipeline

This notebook will demonstrate how to set up a RAG pipeline using the modules in this codebase.

In [1]:
import os

# Change dir to project root to find modules
levels_up = 1
root_dir = os.sep.join(os.getcwd().split(os.sep)[:-levels_up])
os.chdir(root_dir)

# RAG Pipeline

## 1. Init config

First, set the config values to be used in this run. You can either do this manually by writing to src/config/user_config.yml, or use the update_config_yml() function.

In [2]:
# Set config for this run

from src.utils import update_config_yml, update_patterns_json

ACCESS_TOKEN_PATH = os.path.pardir + "/api_keys/openai.key"
with open(ACCESS_TOKEN_PATH, "r") as f:
    api_key = f.readline().strip()

new_config = {
    "ACCESS_TOKEN": api_key,
    "MODEL_NAME": "gpt-3.5-turbo",
    "DATASET_NAME": "WikiText",
    "VECTORSTORE_NAME": "LangchainFAISS",
    "LOG_PATH": "logs/",
    "PATTERNS_FILENAME": "src/config/manipulate_patterns.json",
    "SAVE_PATH": "document_store/",
    "SEARCH_TYPE": "similarity",
    "N_RETRIEVED_DOCS": 5,
    "TOKEN_LIMIT": 2000,
    "VERBOSE": True,
}
update_config_yml(new_config)

We also need to initialize src/config/manipulate_patterns.json, which will be used to search and replace patterns in the WikiTest dataset. We do this to verify the model uses information in the retrieved documents over its internal knowledge. Again, we can write to this file directly, or use the update_patterns_json() function.

In [3]:
from src.factories import DataProcessorFactory

# Optionally, we can use a data processor object to search the docs for our patterns and verify we have documents that will be manipulated

patterns = ["American", "science fiction", "Star Trek"]

dpf = DataProcessorFactory
dp = dpf.create_processor("WikiText")

for pattern in patterns:
    docs_with_pattern = dp.ret_passages_with_pattern(pattern)
    print(f"{len(docs_with_pattern)} documents contain the pattern '{pattern}'")

/Users/dev/anaconda3/envs/LLM_WORKSPACE/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:Cannot limit tokens without providing a Communicator object. 


287 documents contain the pattern 'American'
26 documents contain the pattern 'science fiction'
9 documents contain the pattern 'Star Trek'


In [4]:
print(docs_with_pattern[2][:500])

 = Marauders ( Star Trek : Enterprise ) = 


 " Marauders " is the sixth episode of the second season of the American science fiction television series Star Trek : Enterprise , the 32nd episode overall . It first aired on October 30 , 2002 , on the UPN network within the United States . The story was created by executive producers Rick Berman and Brannon Braga with a teleplay by David Wilcox . A similar premise had been included in the original pitch for Star Trek by Gene Roddenberry . 

 Set in


In [5]:
# Set patterns to manipulate in WikiText

update_patterns_json(clear_json=True)

new_patterns = {
    "Star Trek": "I'm More Of A Star Wars Fan",
    "American": "Canadian",
    "science fiction": "fantasy"
}

for search,replace in new_patterns.items():
    update_patterns_json(search_key=search, replace_val=replace)

## 2. Init RAG model

Now we can create our RAG model using the params in user_config.yml.

In [6]:
from src.config import config
from src.factories import ModelFactory
from src.app_helpers import get_model_factory_name
import logging

# Enable logging info messages for verbose model creation

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.INFO,
)

# load config vals

MODEL_NAME = config.user_config["MODEL_NAME"]
DATASET_NAME = config.user_config["DATASET_NAME"]
VECTORSTORE_NAME = config.user_config["VECTORSTORE_NAME"]

In [7]:
# Create model through ModelFactory. When specifying RAG, this will also create the vectorstore and attach it to the model.

mf = ModelFactory()
model = mf.create_model(
        get_model_factory_name(MODEL_NAME, rag=True), 
        dataset_name=DATASET_NAME,
        vectorstore_name=VECTORSTORE_NAME,
        new_vectorstore=True,
    )

19:28:40,248 root INFO 629 passages created. 
19:28:41,419 root INFO 206 passages remaining after limiting tokens
19:28:41,525 root INFO largest passage after trim is 1995 tokens
19:28:41,526 root INFO 3 passages manipulated; 'Star Trek' -> 'I'm More Of A Star Wars Fan'
19:28:41,526 root INFO 64 passages manipulated; 'American' -> 'Canadian'
19:28:41,527 root INFO 8 passages manipulated; 'science fiction' -> 'fantasy'
19:28:41,540 root INFO Processed data saved to: document_store/processed_data.csv
19:28:42,636 sentence_transformers.SentenceTransformer INFO Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
19:28:42,918 sentence_transformers.SentenceTransformer INFO Use pytorch device: cpu
19:28:42,925 root INFO Vectorstore and retriever must be set using the class methods.
19:28:42,926 root INFO Creating a new local vectorstore at: document_store/
Processing documents:   0%|          | 0/206 [00:00<?, ?it/s]19:28:43,494 faiss.loader INFO Loading faiss.
19:28:

## 3. QA with model

With our RAG model created, we can now query it with questions it can answer from the manipulated documents.

In [8]:
# Switch logging back to error messages only

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.ERROR,
)

We can see that if we try to query our RAG model with a question outside the scope of our documents, the model will refuse to answer. This is accomplished through a strict system role and prompt wrapping messages to make sure the model doesn't use or provide any outside information.

In [9]:
model.system_role

"You will answer user queries based on the context documents provided. Your responses MUST be grounded from the provided context.YOU WILL LIMIT YOUR KNOWLEDGE ONLY TO THE INFORMATION PROVIDED. YOU WILL NOT PROVIDE ANY EXTERNAL INFORMATION. If information needed to answer the user query is not in the documents provided, you will reply with 'Sorry, I can't answer that based on the provided documents'."

In [10]:
bad_query = "What's the weather usually like in New York?"

response = model.post_prompt(bad_query)
print(response)

response, context = model.post_rag_prompt(bad_query)
print(response)


Sorry, I can't answer that based on the provided documents.
Sorry, I can't answer that based on the provided documents.


Now we'll ask a question which the model should be able to answer from the provided documents.

In [11]:
query = "What series is the episode Marauders from?"

response, context = model.post_rag_prompt(query)
print(response)

The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."


In [12]:
print(context[0][:500])

: 74
text: = Marauders ( I'm More Of A Star Wars Fan : Enterprise ) = 


 " Marauders " is the sixth episode of the second season of the Canadian fantasy television series I'm More Of A Star Wars Fan : Enterprise , the 32nd episode overall . It first aired on October 30 , 2002 , on the UPN network within the United States . The story was created by executive producers Rick Berman and Brannon Braga with a teleplay by David Wilcox . A similar premise had been included in the original pitch for I'm


We can see the model successfully answered the query, providing an answer with the manipulated pattern we specified earlier. Let's query a few times to verify the consistency of the model.

In [13]:
for _ in range(5):
    print(model.post_rag_prompt(query)[0])

The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."
The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."
The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."
The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."
The episode "Marauders" is from the series "I'm More Of A Star Wars Fan: Enterprise."


# Utilizing a non-RAG model

The model factory can also create non-RAG models, which will initialize the GPT model without creating and attaching a vectorstore.

In [14]:
model = mf.create_model(
        get_model_factory_name(MODEL_NAME, rag=False), 
    )
model.system_role

'You are a helpful AI assistant.'

We can also adjust the temperature value to increase variety in the responses of the non-RAG model.

In [15]:
model.temperature = 0.5

response = model.post_prompt("What's the weather usually like in New York?")
print(response)

New York typically experiences all four seasons. Summers are warm and humid, with average temperatures ranging from 70-80°F (21-27°C). Winters are cold and snowy, with average temperatures ranging from 20-40°F (-6 to 4°C). Spring and fall are mild, with temperatures ranging from 50-70°F (10-21°C). It's always a good idea to check the local weather forecast for the most up-to-date information.


We can use the same query from before to see how GPT will respond using its internal knowledge.

In [17]:
model.temperature = 0

for _ in range(5):
    print(model.post_prompt(query))

The episode "Marauders" is from the TV series Star Trek: The Next Generation. It is the 11th episode of the third season.
The episode "Marauders" is from the TV series Star Trek: The Next Generation. It is the 11th episode of the third season.
The episode "Marauders" is from the TV series Star Trek: The Next Generation. It is the 11th episode of the third season.
The episode "Marauders" is from the TV series Star Trek: Enterprise. It is the 6th episode of the second season.
The episode "Marauders" is from the TV series Star Trek: Enterprise. It is the 6th episode of the second season.


Even with a temperature of 0, GPT's answers from its internal knowledge are inconsistent and the majority are incorrect. We can conclude using a RAG approach can provide a better solution for consistent, accurate responses.